In [16]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os

# URL of the Reddit page with blacklisted websites
url = "https://www.reddit.com/r/gundeals/wiki/blacklisted_websites/"

# Function to scrape data from the URL
headers = {'User-Agent': 'Mozilla/5.0'}
def getdata(url):
    r = requests.get(url, headers=headers)
    return r.text

# Fetch the HTML data and parse it
htmldata = getdata(url)
soup = BeautifulSoup(htmldata, 'html.parser')

# Extract the relevant section of the HTML
start_collecting = False
blacklisted_sites = []

for tr in soup.find_all('tr'):
    if start_collecting:
        tds = tr.find_all('td')
        if len(tds) > 1:
            site = tds[1].get_text().strip()
            if site and '.' in site and ' ' not in site:
                blacklisted_sites.append(site)
    if "Dealer Website" in str(tr):
        start_collecting = True
    if "Scam Websites" in str(tr):
        break

# Add static sites
static_sites = ["CenterFireGuns.com", "Arkansasgundealer.com"]
blacklisted_sites.extend(static_sites)

# Format the list for uBlock
unique_urls = list(set(blacklisted_sites))  # Remove duplicates

# Prepare the header
header = """[Adblock Plus 2.0]
! Version: 
! Title: Bad Ammo Sites
! Last modified: {date}
! Expires: 365 days (update frequency)
! Homepage: https://github.com/bluesky509/denylist
! This URL: https://raw.githubusercontent.com/bluesky509/clickbait/master/badammo.txt
! License: https://github.com/bluesky509/clickbait/blob/master/LICENSE.txt
""".format(date=datetime.now().strftime("%Y-%m-%d"))

# Save the formatted list to a file in the same directory
output_path = 'ublock_blacklist.txt'

with open(output_path, 'w') as file:
    file.write(header)
    for site in unique_urls:
        file.write(f"{site}\n")

print(f"Blacklist saved to {output_path}")

Blacklist saved to ublock_blacklist.txt
